In [2]:
!pip install mediapipe opencv-python pandas scikit-learn

In [10]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv

In [11]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

In [14]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [15]:
results.face_landmarks.landmark[0].visibility

0.0

In [75]:
import csv
import os
import numpy as np

In [76]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [81]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [88]:
class_name = "Love"

In [89]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 

            # # Make Detections
            # X = pd.DataFrame([row])
            # body_language_class = model.predict(X)[0]
            # body_language_prob = model.predict_proba(X)[0]
            # print(body_language_class, body_language_prob)
            
            # # Grab ear coords
            # coords = tuple(np.multiply(
            #                 np.array(
            #                     (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
            #                      results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
            #             , [640,480]).astype(int))
            
            # cv2.rectangle(image, 
            #               (coords[0], coords[1]+5), 
            #               (coords[0]+len(body_language_class)*20, coords[1]-30), 
            #               (245, 117, 16), -1)
            # cv2.putText(image, body_language_class, coords, 
            #             cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # # Get status box
            # cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # # Display Class
            # cv2.putText(image, 'CLASS'
            #             , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            # cv2.putText(image, body_language_class.split(' ')[0]
            #             , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # # Display Probability
            # cv2.putText(image, 'PROB'
            #             , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            # cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
            #             , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [48]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [49]:
df = pd.read_csv('coords.csv')

In [50]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Happy,0.467039,0.566748,-1.280697,0.996546,0.504434,0.483972,-1.213821,0.993556,0.528113,...,-0.031150,0.0,0.530514,0.478321,-0.024694,0.0,0.536181,0.467887,-0.025233,0.0
1,Happy,0.461606,0.564403,-0.888078,0.996866,0.497059,0.483925,-0.862152,0.994146,0.519717,...,-0.025823,0.0,0.526703,0.484282,-0.018223,0.0,0.531840,0.475909,-0.018554,0.0
2,Happy,0.455548,0.563694,-0.803549,0.997158,0.487411,0.483889,-0.778104,0.994682,0.509205,...,-0.027032,0.0,0.525604,0.485038,-0.020079,0.0,0.530991,0.476595,-0.020432,0.0
3,Happy,0.452990,0.564032,-0.915711,0.997427,0.482932,0.484927,-0.887372,0.995175,0.503929,...,-0.026971,0.0,0.526447,0.487042,-0.019924,0.0,0.531740,0.478359,-0.020260,0.0
4,Happy,0.452041,0.564543,-0.910424,0.997671,0.481368,0.485950,-0.887595,0.995619,0.502112,...,-0.026402,0.0,0.525802,0.485933,-0.019648,0.0,0.531156,0.477041,-0.019947,0.0


In [51]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
607,Love,0.422996,0.582429,-1.447696,0.999938,0.449061,0.507325,-1.437020,0.999799,0.469658,...,-0.020946,0.0,0.490197,0.491845,-0.014403,0.0,0.495242,0.484075,-0.014916,0.0
608,Love,0.421226,0.581926,-1.403193,0.999936,0.447425,0.506844,-1.391837,0.999794,0.468132,...,-0.020780,0.0,0.489203,0.490609,-0.014989,0.0,0.494324,0.482783,-0.015553,0.0
609,Love,0.420940,0.580474,-1.402862,0.999936,0.447198,0.505766,-1.388709,0.999793,0.467945,...,-0.020765,0.0,0.489056,0.487745,-0.014396,0.0,0.494012,0.480025,-0.014961,0.0
610,Love,0.420626,0.579636,-1.399485,0.999936,0.446942,0.505136,-1.384689,0.999794,0.467737,...,-0.021016,0.0,0.488487,0.488927,-0.014698,0.0,0.493538,0.481330,-0.015255,0.0
611,Love,0.420545,0.579015,-1.456003,0.999937,0.446815,0.504681,-1.435905,0.999794,0.467624,...,-0.020927,0.0,0.488936,0.490110,-0.014655,0.0,0.494059,0.481701,-0.015119,0.0


In [54]:
X = df.drop('class', axis=1) # features
y = df['class'] #target value

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [56]:
y_train

47          Happy
206           Sad
13          Happy
89          Happy
22          Happy
          ...    
279           Sad
372    Victorious
204           Sad
53          Happy
294           Sad
Name: class, Length: 428, dtype: object

In [57]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [58]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [59]:
pipelines.keys()

dict_keys(['lr', 'rc', 'rf', 'gb'])

In [61]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

C:\Users\Allon\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [62]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [63]:
fit_models['rc'].predict(X_test)

array(['Sad', 'Sad', 'Love', 'Victorious', 'Love', 'Sad', 'Sad', 'Happy',
       'Victorious', 'Victorious', 'Love', 'Sad', 'Sad', 'Victorious',
       'Sad', 'Sad', 'Sad', 'Happy', 'Sad', 'Victorious', 'Love', 'Happy',
       'Sad', 'Love', 'Sad', 'Victorious', 'Happy', 'Love', 'Happy',
       'Happy', 'Victorious', 'Love', 'Love', 'Sad', 'Love', 'Happy',
       'Love', 'Sad', 'Happy', 'Victorious', 'Victorious', 'Victorious',
       'Victorious', 'Happy', 'Sad', 'Love', 'Love', 'Victorious',
       'Victorious', 'Sad', 'Sad', 'Love', 'Sad', 'Happy', 'Sad', 'Love',
       'Love', 'Love', 'Love', 'Love', 'Happy', 'Happy', 'Happy', 'Sad',
       'Sad', 'Sad', 'Happy', 'Sad', 'Love', 'Sad', 'Sad', 'Sad', 'Happy',
       'Sad', 'Happy', 'Victorious', 'Sad', 'Sad', 'Happy', 'Happy',
       'Sad', 'Happy', 'Victorious', 'Victorious', 'Victorious', 'Love',
       'Happy', 'Victorious', 'Sad', 'Victorious', 'Love', 'Sad', 'Happy',
       'Love', 'Sad', 'Love', 'Sad', 'Love', 'Sad', 'Love', 'V

In [64]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

In [65]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 0.9945652173913043
rf 1.0
gb 0.9836956521739131


In [66]:
fit_models['rf'].predict(X_test)

array(['Sad', 'Sad', 'Love', 'Victorious', 'Love', 'Sad', 'Sad', 'Happy',
       'Victorious', 'Victorious', 'Love', 'Sad', 'Sad', 'Victorious',
       'Sad', 'Sad', 'Sad', 'Happy', 'Sad', 'Victorious', 'Love', 'Happy',
       'Sad', 'Love', 'Sad', 'Victorious', 'Happy', 'Love', 'Happy',
       'Happy', 'Victorious', 'Love', 'Love', 'Sad', 'Love', 'Happy',
       'Love', 'Sad', 'Happy', 'Victorious', 'Victorious', 'Victorious',
       'Victorious', 'Happy', 'Sad', 'Love', 'Love', 'Victorious',
       'Victorious', 'Sad', 'Sad', 'Love', 'Sad', 'Happy', 'Sad', 'Love',
       'Love', 'Love', 'Love', 'Love', 'Happy', 'Happy', 'Happy', 'Sad',
       'Sad', 'Sad', 'Happy', 'Sad', 'Love', 'Sad', 'Sad', 'Sad', 'Happy',
       'Sad', 'Happy', 'Victorious', 'Sad', 'Sad', 'Happy', 'Happy',
       'Sad', 'Happy', 'Victorious', 'Victorious', 'Victorious', 'Love',
       'Happy', 'Victorious', 'Sad', 'Victorious', 'Love', 'Sad', 'Happy',
       'Love', 'Sad', 'Love', 'Sad', 'Love', 'Sad', 'Love', 'V

In [67]:
y_test

163           Sad
289           Sad
566          Love
335    Victorious
530          Love
          ...    
93          Happy
185           Sad
467          Love
505          Love
607          Love
Name: class, Length: 184, dtype: object

In [68]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

In [69]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [70]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [90]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
#             # Append class name 
#             row.insert(0, class_name)
            
#             # Export to CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row) 

            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

C:\Users\Allon\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Allon\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Allon\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Allon\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Happy [0.66 0.06 0.28 0.  ]
Happy [0.89 0.1  0.01 0.  ]


C:\Users\Allon\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Allon\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Allon\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Allon\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Happy [0.92 0.07 0.01 0.  ]
Happy [0.93 0.06 0.01 0.  ]


C:\Users\Allon\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Allon\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Allon\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Allon\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


...

Sad [0.25 0.03 0.68 0.04]
Sad [0.24 0.03 0.67 0.06]


C:\Users\Allon\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Allon\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Allon\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Allon\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Sad [0.24 0.03 0.67 0.06]
Sad [0.23 0.04 0.67 0.06]
Sad [0.23 0.04 0.67 0.06]


C:\Users\Allon\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Allon\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


KeyboardInterrupt: 

In [71]:
tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)), [640,480]).astype(int))

(331, 242)